In [1]:
import pandas as pd
import os

In [2]:
match_directory = 'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\matches'
location_directory = 'C:\\Users\\Alec\\Documents\\Python\\soccer-ai\\data_files\\venue_location_data'
# core_cols = mega_match_file.drop(columns=['home_player_stats', 'away_player_stats', 'match_events', 'officials', 'shots', 'overall_stats'])
core_cols = ['away_npxG', 'away_team', 'home_goals', 'home_team', 'home_xAG', 'home_team_id', 'league', 'utc_datetime', 'home_ast', 'local_date', 'away_team_id', 'link', 'away_formation', 'away_ast', 'home_manager',
             'stage', 'year', 'attendance', 'away_xAG', 'second_half_stoppage', 'away_goals', 'home_npxG', 'away_manager', 'first_half_stoppage', 'home_captain', 'away_captain', 'home_formation', 'home_xG', 'away_xG', 'venue']
mega_venue_file = pd.read_parquet(location_directory + '\\all_venue_data.parquet')
match_data_file_list = [data_file for data_file in os.listdir(match_directory)
                        if '.pkl' in data_file]

In [3]:
match_data_file_list

['Bundesliga 2_FBRef_scrape.pkl',
 'Bundesliga_FBRef_scrape.pkl',
 'EFL Championship_FBRef_scrape.pkl',
 'EPL_FBRef_scrape.pkl',
 'Eredivisie_FBRef_scrape.pkl',
 'final_8_fails.pkl',
 'five_x_fail_final2_FBRef_scrape.pkl',
 'La Liga 2_FBRef_scrape.pkl',
 'La Liga_FBRef_scrape.pkl',
 'Ligue 1_FBRef_scrape.pkl',
 'Ligue 2_FBRef_scrape.pkl',
 'missing_matches_FBRef_scrape.pkl',
 'Primeira Liga_FBRef_scrape.pkl',
 'Serie A_FBRef_scrape.pkl',
 'Serie B_FBRef_scrape.pkl']

In [5]:
test = pd.read_pickle(match_directory + '\\' + match_data_file_list[1])

In [6]:

def pull_home_away_stats(match_info,):
    cols = ['home_player_stats', 'away_player_stats']

    out_stats = pd.DataFrame()
    for stats_col in cols:
        _stat_mask = match_info[stats_col].apply(lambda x: (x.shape[1] > 1) if x is not None else False)
        _stat_level = match_info.loc[_stat_mask, stats_col].apply(lambda x: x.iloc[0])
        _stat_level['link'] = match_info.loc[_stat_mask, 'link']
        _stat_level['local_date'] = match_info.loc[_stat_mask, 'local_date']
        _stat_level['side'] = stats_col[:4]
        _stat_level['team'] = match_info.loc[_stat_mask, stats_col[:5] + 'team']
        out_stats = pd.concat([out_stats, _stat_level], ignore_index=True)

    out_stats = out_stats.sort_values('local_date', ascending=False).reset_index(drop=True)
    return out_stats


def access_stats_objects(extract_cols: list[str], stat_info: pd.DataFrame,) -> dict[str, pd.DataFrame]:
    out_frames = {}

    for i, col in enumerate(extract_cols):
        col_items = stat_info[col].items()
        out_df = pd.DataFrame()

        if col == 'bench':
            for i, item in enumerate(col_items):
                if item[1] is not None:
                    tmp_out = item[1].to_frame().T
                    tmp_out['link'], tmp_out['local_date'], tmp_out['formation'], tmp_out['team'], tmp_out['side'] = stat_info.loc[item[0],
                                                                                                                                   'link'], stat_info.loc[item[0], 'local_date'], stat_info.loc[item[0], 'formation'], stat_info.loc[item[0], 'team'], stat_info.loc[item[0], 'side']
                    out_df = pd.concat([out_df, tmp_out], ignore_index=True)
        elif col == 'starting_lineup':
            for i, item in enumerate(col_items):
                if item[1] is not None:
                    tmp_out = item[1]
                    tmp_out['link'], tmp_out['local_date'], tmp_out['formation'], tmp_out['team'], tmp_out['side'] = stat_info.loc[item[0],
                                                                                                                                   'link'], stat_info.loc[item[0], 'local_date'], stat_info.loc[item[0], 'formation'], stat_info.loc[item[0], 'team'], stat_info.loc[item[0], 'side']
                    out_df = pd.concat([out_df, tmp_out], ignore_index=True)
        else:
            for i, item in enumerate(col_items):
                if item[1] is not None:
                    tmp_out = item[1]
                    tmp_out['link'], tmp_out['local_date'], tmp_out['team'], tmp_out['side'] = stat_info.loc[item[0],
                                                                                                             'link'], stat_info.loc[item[0], 'local_date'], stat_info.loc[item[0], 'team'], stat_info.loc[item[0], 'side']
                    out_df = pd.concat([out_df, tmp_out], ignore_index=True)

        out_df = out_df.sort_values('local_date', ascending=False).reset_index(drop=True)
        out_frames[col] = out_df
    return out_frames

In [7]:
stat_info = pull_home_away_stats(test)

In [ ]:
pd.Series()

In [24]:
cur_col_mask = stat_info['passing'].notnull()
g = pd.concat([
    pd.concat([tup[1], pd.Series(tup[0], name='orig_index', index=range(tup[1].shape[0]))], axis=1) for tup in stat_info.loc[cur_col_mask, 'passing'].items() if tup is not None], ignore_index=True)

In [29]:
g = pd.concat([
    pd.concat([tup[1], pd.Series(tup[0], name='orig_index', index=range(tup[1].shape[0]))], axis=1) for tup in stat_info.loc[cur_col_mask, 'passing'].items() if tup is not None], ignore_index=True)

In [34]:
g[g['orig_index'] == 3899]

,"(Unnamed: 0_level_0, Player)","(Unnamed: 1_level_0, #)","(Unnamed: 2_level_0, Nation)","(Unnamed: 3_level_0, Pos)","(Unnamed: 4_level_0, Age)","(Unnamed: 5_level_0, Min)","(Total, Cmp)","(Total, Att)","(Total, Cmp%)","(Total, TotDist)",...,"(Unnamed: 20_level_0, Ast)","(Unnamed: 21_level_0, xAG)","(Unnamed: 22_level_0, xA)","(Unnamed: 23_level_0, KP)","(Unnamed: 24_level_0, 1/3)","(Unnamed: 25_level_0, PPA)","(Unnamed: 26_level_0, CrsPA)","(Unnamed: 27_level_0, PrgP)","(player_id, )",orig_index
61351,Robert Lewandowski,9.0,pl POL,FW,10582.0,53,20.0,25.0,80.0,304.0,...,0,0.3,0.1,2.0,2.0,2.0,0.0,4.0,8d78e732,3899
61352,Franck Ribéry,7.0,fr FRA,LW,12543.0,77,33.0,35.0,94.3,653.0,...,0,0.5,0.2,2.0,3.0,1.0,0.0,9.0,f61c7c4e,3899
61353,Kingsley Coman,29.0,fr FRA,LW,7731.0,14,7.0,8.0,87.5,110.0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,042e8a49,3899
61354,Thomas Müller,25.0,de GER,RW,10194.0,61,12.0,17.0,70.6,211.0,...,0,0.1,0.2,1.0,0.0,1.0,0.0,1.0,3c6089ab,3899
61355,Arjen Robben,10.0,nl NED,RW,12252.0,30,12.0,16.0,75.0,183.0,...,0,0.3,0.0,1.0,1.0,0.0,0.0,2.0,09586899,3899
61356,Arturo Vidal,23.0,cl CHI,LM,11038.0,91,48.0,59.0,81.4,1004.0,...,1,0.3,0.8,1.0,7.0,1.0,0.0,7.0,67cdbc0e,3899
61357,Sebastian Rudy,19.0,de GER,CM,10026.0,91,43.0,48.0,89.6,814.0,...,1,0.2,0.2,2.0,6.0,1.0,0.0,5.0,80f36c88,3899
61358,Corentin Tolisso,24.0,fr FRA,RM,8410.0,91,49.0,63.0,77.8,1068.0,...,0,0.0,0.2,0.0,9.0,2.0,0.0,12.0,652d4c37,3899
61359,David Alaba,27.0,at AUT,LB,9180.0,91,55.0,64.0,85.9,1060.0,...,0,0.0,0.1,0.0,6.0,3.0,2.0,4.0,05439de2,3899
61360,Mats Hummels,5.0,de GER,CB,10465.0,62,44.0,50.0,88.0,1016.0,...,0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,05d548d8,3899


In [32]:
url_map = stat_info['link'].to_dict()

In [37]:
g['link'] = g['orig_index'].map(url_map)

In [38]:
g

,"(Unnamed: 0_level_0, Player)","(Unnamed: 1_level_0, #)","(Unnamed: 2_level_0, Nation)","(Unnamed: 3_level_0, Pos)","(Unnamed: 4_level_0, Age)","(Unnamed: 5_level_0, Min)","(Total, Cmp)","(Total, Att)","(Total, Cmp%)","(Total, TotDist)",...,"(Unnamed: 21_level_0, xAG)","(Unnamed: 22_level_0, xA)","(Unnamed: 23_level_0, KP)","(Unnamed: 24_level_0, 1/3)","(Unnamed: 25_level_0, PPA)","(Unnamed: 26_level_0, CrsPA)","(Unnamed: 27_level_0, PrgP)","(player_id, )",orig_index,link
0,Omar Marmoush,7.0,eg EGY,FW,9059.0,96,17.0,26.0,65.4,220.0,...,0.1,0.1,1.0,2.0,1.0,0.0,3.0,0e0102eb,0,https://fbref.com/en/matches/4bea1c30/Augsburg...
1,Ansgar Knauff,36.0,de GER,LW,7992.0,69,13.0,21.0,61.9,202.0,...,0.0,0.1,1.0,0.0,1.0,1.0,2.0,19a967ab,0,https://fbref.com/en/matches/4bea1c30/Augsburg...
2,Mario Götze,27.0,de GER,RM,11498.0,33,32.0,35.0,91.4,420.0,...,0.0,0.0,0.0,0.0,1.0,0.0,4.0,9efa1d25,0,https://fbref.com/en/matches/4bea1c30/Augsburg...
3,Aurélio Buta,24.0,pt POR,RW,9786.0,51,28.0,34.0,82.4,351.0,...,0.0,0.1,0.0,1.0,2.0,1.0,2.0,b4705807,0,https://fbref.com/en/matches/4bea1c30/Augsburg...
4,Niels Nkounkou,29.0,fr FRA,RW,8427.0,51,15.0,21.0,71.4,210.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,f5b281e7,0,https://fbref.com/en/matches/4bea1c30/Augsburg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61361,Rafinha,13.0,br BRA,CB,11660.0,29,18.0,22.0,81.8,324.0,...,0.0,0.0,1.0,2.0,0.0,0.0,1.0,d003e213,3899,https://fbref.com/en/matches/b13970b2/Bayern-M...
61362,Niklas Süle,4.0,de GER,CB,8014.0,91,43.0,50.0,86.0,875.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,e33c06cf,3899,https://fbref.com/en/matches/b13970b2/Bayern-M...
61363,Joshua Kimmich,32.0,de GER,RB,8221.0,91,44.0,53.0,83.0,811.0,...,0.0,0.1,0.0,2.0,2.0,0.0,3.0,49296448,3899,https://fbref.com/en/matches/b13970b2/Bayern-M...
61364,Sven Ulreich,26.0,de GER,GK,10600.0,91,31.0,37.0,83.8,974.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7d3e7da6,3899,https://fbref.com/en/matches/b13970b2/Bayern-M...
